<a href="https://colab.research.google.com/github/SpectreB/Data-Sci/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Preparing Data

# Import Statements
import pandas as pd
import numpy as np

# Data visualization
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms
from sklearn.linear_model import LogisticRegression as LogRegression
from sklearn import linear_model as LinModel


In [2]:
Gender=pd.read_csv("/content/gender_submission.csv")
test_data=pd.read_csv("/content/test.csv")
train_data=pd.read_csv("/content/train.csv")

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
train_data.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0,NaN,S
253,254,0,3,"Lobb, Mr. William Arthur",male,30.0,1,0,A/5. 3336,16.1,NaN,S
56,57,1,2,"Rugg, Miss. Emily",female,21.0,0,0,C.A. 31026,10.5,NaN,S
822,823,0,1,"Reuchlin, Jonkheer. John George",male,38.0,0,0,19972,0.0,NaN,S
714,715,0,2,"Greenberg, Mr. Samuel",male,52.0,0,0,250647,13.0,NaN,S


In [ ]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [ ]:
train_data.columns.values


array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [5]:
#Looking at the data to fix (tf)

Total_tf = train_data.isnull().sum().sort_values(ascending=False)
Percent_V1 = train_data.isnull().sum()/train_data.isnull().count()*100

Percent_V2 = (round(Percent_V1, 1)).sort_values(ascending=False)

data_to_fix = pd.concat([Total_tf, Percent_V2], axis=1, keys=['Total', 'Percentage'])
data_to_fix
data_to_fix.head(3)

,Total,Percentage
Cabin,687,77.1
Age,177,19.9
Embarked,2,0.2


In [6]:
train_data['Embarked'].describe()# to find the most common value, which is S


count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [7]:
#DATA PREPROCESSING


#Missing Data



train_data = train_data.drop(['PassengerId'], axis=1) #not missing anything but useless for our purpose
train_data = train_data.drop(['Cabin'], axis=1)

train_data = train_data.drop(['Ticket'], axis=1)
test_data = test_data.drop(['Ticket'], axis=1)



  #Age

tot_data= [train_data, test_data]

for dataset in tot_data:
    mean = train_data["Age"].mean()
    std = test_data["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_data["Age"].astype(int)
train_data["Age"].isnull().sum()

  #Embarked
for dataset in tot_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')



In [8]:

ports = {"S": 0, "C": 1, "Q": 2}

for dataset in tot_data:
    dataset['Embarked'] = dataset['Embarked'].map(ports)


In [ ]:
train_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Close_relatives,not_alone,Age_Class,Fare_Per_Person,Title
0,0,3,NaN,0,1,0,0,0,1,0,0,0,1
1,1,1,NaN,0,1,0,0,1,1,0,0,0,3
2,1,3,NaN,0,0,0,0,0,0,1,0,0,2
3,1,1,NaN,0,1,0,0,0,1,0,0,0,3
4,0,3,NaN,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,NaN,0,0,0,0,0,0,1,0,0,5
887,1,1,NaN,0,0,0,0,0,0,1,0,0,2
888,0,3,NaN,0,1,2,0,0,3,0,0,0,2
889,1,1,NaN,0,0,0,0,1,0,1,0,0,1


In [9]:
#DATA PREPROCESSING 2

  #Converting Features
  #Fare (converting it to int)

for dataset in tot_data:
    dataset['Fare'] = dataset['Fare'].fillna(0)
    dataset['Fare'] = dataset['Fare'].astype(int)

  #Sex

genders = {"male": 0, "female": 1}
for dataset in tot_data:
    dataset['Sex'] = dataset['Sex'].map(genders)

  #Name

titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}


for dataset in tot_data:
    # extract titles
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)

train_data = train_data.drop(['Name'], axis=1)
test_data = test_data.drop(['Name'], axis=1)



  #Embarked
ports = {"S": 0, "C": 1, "Q": 2}

for dataset in tot_data:
    dataset['Embarked'] = dataset['Embarked'].map(ports)



In [11]:
 #Age - Creating Close_Relatives category

for dataset in tot_data:
    dataset['Close_relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['Close_relatives'] > 0, 'not_alone'] = 0
    dataset.loc[dataset['Close_relatives'] == 0, 'not_alone'] = 1
    dataset['not_alone'] = dataset['not_alone'].astype(int)




 #Age - Creating Age category

for dataset in tot_data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 7

for dataset in tot_data:
    dataset['Age_Class']= dataset['Age']* dataset['Pclass']

#Fare

dataset['Fare'].fillna(dataset['Fare'].median(), inplace=True)


for dataset in tot_data:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3
    dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4
    dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)


for dataset in tot_data:
    dataset['Fare_Per_Person'] = dataset['Fare']/(dataset['Close_relatives']+1)
    dataset['Fare_Per_Person'] = dataset['Fare_Per_Person'].astype(int)

In [12]:
print(dataset['Fare'].isnull().sum())


0


In [13]:
#Building Machine Learning Models

X_train = train_data
Y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId", axis=1).copy()

In [14]:
#Logistic Regression:


logreg = LogRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

acc_log = round(logreg.score(X_train, Y_train) * 100, 2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: ignored

In [ ]:
X_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Close_relatives,not_alone,Age_Class,Fare_Per_Person,Title
0,0,3,NaN,0,1,0,0,NaN,1,0,0,0,1
1,1,1,NaN,0,1,0,0,NaN,1,0,0,0,3
2,1,3,NaN,0,0,0,0,NaN,0,1,0,0,2
3,1,1,NaN,0,1,0,0,NaN,1,0,0,0,3
4,0,3,NaN,0,0,0,0,NaN,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,NaN,0,0,0,0,NaN,0,1,0,0,5
887,1,1,NaN,0,0,0,0,NaN,0,1,0,0,2
888,0,3,NaN,0,1,2,0,NaN,3,0,0,0,2
889,1,1,NaN,0,0,0,0,NaN,0,1,0,0,1
